# Ray Serve - Model Serving Challenges

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

## Challenges of Model Serving

Model development happens in a data science research environment. There are many challenges, but also tools at the data scientists disposal.

Model deployment to production faces an entirely different set of challenges and requires different tools, although it is desirable to bridge the divide as much as possible.

Here is a partial lists of the challenges of model serving:

### It Should Be Framework Agnostic

Model serving frameworks must be able to serve models from popular systems like TensorFlow, PyTorch, scikit-learn, or even arbitrary Python functions. Even within the same organization, it is common to use several machine learning frameworks. 

Also, machine learning models are typically surrounded by lots of application logic. For example, some model serving is implemented as a RESTful service to which scoring requests are made. Often this is too restrictive, as some additional processing may be desired as part of the scoring process, and the performance overhead of remote calls may be suboptimal.

### Pure Python

It has been common recently for model serving to be done using JVM-based systems, since many production enterprises are JVM-based. This is a disadvantage when model training and other data processing are done using Python tools, only. 

In general, model serving should be intuitive for developers and simple to configure and run. Hence, it is desirable to use pure Python and to avoid verbose configurations using YAML files or other means. 

Data scientists and engineers use Python to develop their machine learning models, so they should also be able to use Python to deploy their machine learning applications. This need is growing more critical as online learning applications combine training and serving in the same applications.

### Simple and Scalable

Model serving must be simple to scale on demand across many machines. It must also be easy to upgrade models dynamically, over time. Achieving production uptime and performance requirements are essential for success.

### DevOps Integrations

Model serving deployments need to integrate with existing "DevOps" CI/CD practices for controlled, audited, and predicatble releases. Patterns like [Canary Releases](https://martinfowler.com/bliki/CanaryRelease.html) are particularly useful for testing the efficacy of a new model before replacing existing models, just as this pattern is useful for other software deployments.

### Flexible Deployment Patterns

There are unique deployment patterns, too. For example, it should be easy to deploy a forest of models, to split traffic to different instances, and to score data in batches for greater efficiency.

See also this [Ray blog post](https://medium.com/distributed-computing-with-ray/the-simplest-way-to-serve-your-nlp-model-in-production-with-pure-python-d42b6a97ad55) on the challenges of model serving and the way Ray Serve addresses them. It also provides an example of starting with a simple model, then deploying a more sophisticated model into the running application.

## Why Ray Serve?

[Ray Serve](https://docs.ray.io/en/latest/serve/index.html) is a scalable model-serving library built on [Ray](https://ray.io).

For users, Ray Serve offers these benefits:

* **Framework Agnostic**: You can use the same toolkit to serve everything from deep learning models built with [PyTorch](https://docs.ray.io/en/latest/serve/tutorials/pytorch.html#serve-pytorch-tutorial), [Tensorflow](https://docs.ray.io/en/latest/serve/tutorials/tensorflow.html#serve-tensorflow-tutorial), or [Keras](https://docs.ray.io/en/latest/serve/tutorials/tensorflow.html#serve-tensorflow-tutorial), to [scikit-Learn](https://docs.ray.io/en/latest/serve/tutorials/sklearn.html#serve-sklearn-tutorial) models, to arbitrary business logic.
* **Python First:** Configure your model serving with pure Python code. No YAML or JSON configurations required.

As a library, Ray Serve enables the following:

* [Splitting traffic between backends dynamically](https://docs.ray.io/en/latest/serve/advanced.html#serve-split-traffic) with zero downtime. This is accomplished by decoupling routing logic from response handling logic.
* [Support for batching](https://docs.ray.io/en/latest/serve/advanced.html#serve-batching) to improve performance helps you meet your performance objectives. You can also use a model for batch and online processing.

Since Serve is built on Ray, it also allows you to scale to many machines, in your datacenter or in cloud environments, and it allows you to leverage all of the other Ray frameworks.

## Two Simple Ray Serve Examples

We'll explore a more detailed example in the next lesson. Here we explore how simple deployments are simple with Ray Serve! We will first use a function that does "scoring", then a class.

But first, initialize Ray as before:

In [1]:
!../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [2]:
import ray
from ray import serve

In [3]:
ray.init(address='auto', ignore_reinit_error=True)

{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:6379',
 'object_store_address': '/tmp/ray/session_2020-07-19_08-56-14_461147_28318/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-19_08-56-14_461147_28318/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-07-19_08-56-14_461147_28318'}

Note that Serve leverages the [Flask API](https://flask.palletsprojects.com/en/1.1.x/api/), which is often familiar, as it is a natural first approach for deploying models as RESTful services.

In [4]:
import requests  # for making web requests

In [5]:
serve.init(name='serve-example-1')  # Name for this Serve instance

2020-07-19 12:26:53,832	WARNING import_thread.py:136 -- The actor 'TrainMNIST' has been exported 100 times. It's possible that this warning is accidental, but this may indicate that the same remote function is being defined repeatedly from within many tasks and exported to all of the workers. This can be a performance issue and can be resolved by defining the remote function on the driver instead. See https://github.com/ray-project/ray/issues/6240 for more discussion.


In [6]:
def echo(flask_request):          # Uses the Flask API 
    return "hello " + flask_request.args.get("name", "serve!")

In [7]:
serve.create_backend("hello", echo)
serve.create_endpoint("hello", backend="hello", route="/hello")

In [8]:
for i in range(10):
    response = requests.get(f"http://127.0.0.1:8000/hello?name=request_{i}").text
    print(f'{i:2d}: {response}')

 0: hello request_0
 1: hello request_1
 2: hello request_2
 3: hello request_3
 4: hello request_4
 5: hello request_5
 6: hello request_6
 7: hello request_7
 8: hello request_8
 9: hello request_9


You should see `hello request_N` in the output. Try making `requests.get()` invocations without the `?name=request_{i}` parameter. You should see `hello serve!`.

We'll explain the concepts of _backends_ and _endpoints_ below. 

Now let's serve another "model" in the same service:

In [9]:
class Counter:
    def __init__(self, initial_count = 0):
        self.count = initial_count

    def __call__(self, flask_request):
        self.count += 1
        return {"current_counter": self.count, "args": flask_request.args}

When we create the _backend_, we can pass constructor arguments after the label and the name of the class:

In [10]:
serve.create_backend("counter", Counter, 0)  # initial_count = 0
serve.create_endpoint("counter", backend="counter", route="/counter")

In [11]:
for i in range(10):
    response = requests.get(f"http://127.0.0.1:8000/counter?i={i}").json()
    print(f'{i:2d}: {response}')

 0: {'current_counter': 21, 'args': {'i': '0'}}
 1: {'current_counter': 22, 'args': {'i': '1'}}
 2: {'current_counter': 23, 'args': {'i': '2'}}
 3: {'current_counter': 24, 'args': {'i': '3'}}
 4: {'current_counter': 25, 'args': {'i': '4'}}
 5: {'current_counter': 26, 'args': {'i': '5'}}
 6: {'current_counter': 27, 'args': {'i': '6'}}
 7: {'current_counter': 28, 'args': {'i': '7'}}
 8: {'current_counter': 29, 'args': {'i': '8'}}
 9: {'current_counter': 30, 'args': {'i': '9'}}


## Exercise - Add Another New Backend and Endpoint

Using either a function or a stateful class, add another _backend_ and _endpoint_, then try it out.

## Ray Serve Concepts

Let's explain _backends_ and _endpoints_.

For more details, see this [key concepts](https://docs.ray.io/en/latest/serve/key-concepts.html) documentation.

### Backends

Backends define the implementation of your business logic or models that will handle requests when queries come in to _endpoints._ 

To define a backend, first define the “handler” or business logic that will take requests and construct responses. Specifically, the handler should take as input a [Flask Request object](https://flask.palletsprojects.com/en/1.1.x/api/?highlight=request#flask.Request) and return any JSON-serializable object as output. 

Use a function when your response is _stateless_ and a class when your response is _stateful_ (although the class instances could be stateless, of course). Another advantage of using a class is the ability to specify constructor arguments in `serve.create_backend`, as was shown in the `counter` example above.

Finally, a backend is defined using `serve.create_backend`, specifying a logical, unique name, and the handler.

You can list all defined backends and delete them to reclaim resources. However, a backend cannot be deleted while it is in use by an endpoint, because then traffic to an endpoint could not be handled:

In [12]:
serve.create_backend("counter_toss", Counter, 0)
serve.list_backends()

{'hello': {'accepts_batches': False,
  'num_replicas': 1,
  'max_batch_size': None},
 'counter': {'accepts_batches': False,
  'num_replicas': 1,
  'max_batch_size': None},
 'counter_toss': {'accepts_batches': False,
  'num_replicas': 1,
  'max_batch_size': None}}

In [13]:
serve.delete_backend("counter_toss")

In [14]:
serve.list_backends()

{'hello': {'accepts_batches': False,
  'num_replicas': 1,
  'max_batch_size': None},
 'counter': {'accepts_batches': False,
  'num_replicas': 1,
  'max_batch_size': None}}

### Endpoints

While a backend defines the request handling logic, an endpoint allows you to expose a backend via HTTP. Endpoints are “logical” and can have one or multiple backends that serve requests to them. 

To create an endpoint, you specify a name for the endpoint, the name of a backend to handle requests to the endpoint, and the route and the list of HTTP methods (e.g., `[GET]`, which is the default) where it will be accesible. By default endpoints are serviced only by the backend provided to `serve.create_endpoint`, but in some cases you may want to specify multiple backends for an endpoint, e.g., for A/B testing or incremental rollout. For information on traffic splitting, please see [Splitting Traffic Between Backends](https://docs.ray.io/en/latest/serve/advanced.html#serve-split-traffic).

Let's define a second endpoint for our `hello` backend, this one providing `POST` access. (We could have defined the original `hello` endpoint to support `POST` and `GET` using `methods = ['POST', 'GET']`.)

In [15]:
serve.create_endpoint("post_hello", backend="hello", route="/post_hello", methods=["POST"])

In [16]:
eds = serve.list_endpoints()
eds.keys(), eds

(dict_keys(['hello', 'counter', 'post_hello']),
 {'hello': {'route': '/hello', 'methods': ['GET'], 'traffic': {'hello': 1.0}},
  'counter': {'route': '/counter',
   'methods': ['GET'],
   'traffic': {'counter': 1.0}},
  'post_hello': {'route': '/post_hello',
   'methods': ['POST'],
   'traffic': {'hello': 1.0}}})

In [ ]:
for i in range(10):
    response = requests.post(f"http://127.0.0.1:8000/post_hello", data = {'name': f'request_{i}'})
    print(f'{i:2d}: {response}')

(pid=32967) 2020-07-19 12:28:03,336	INFO (unknown file):0 -- gc.collect() freed 10 refs in 0.12182619399936812 seconds
(pid=32970) 2020-07-19 12:28:03,328	INFO (unknown file):0 -- gc.collect() freed 50 refs in 0.1284467939995011 seconds
(pid=32965) 2020-07-19 12:28:03,341	INFO (unknown file):0 -- gc.collect() freed 10 refs in 0.09797865200016531 seconds
(pid=35708) 2020-07-19 12:28:03,402	INFO (unknown file):0 -- gc.collect() freed 24 refs in 0.069170064999998 seconds
(pid=35707) 2020-07-19 12:28:03,406	INFO (unknown file):0 -- gc.collect() freed 24 refs in 0.06263057399999994 seconds
(pid=35705) 2020-07-19 12:28:06,177	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/TrainMNIST/TrainMNIST_6_2020-07-19_12-27-44yalgnoh8/tmpa3icubdorestore_from_object/checkpoint
(pid=35705) 2020-07-19 12:28:06,177	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 20, '_timesteps_total': None, '_time_total': 5.945605278015137, '_episod

2020-07-19 12:28:22,480	WARNING worker.py:1047 -- WARNING: 29 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-19 12:28:22,480	WARNING worker.py:1047 -- WARNING: 30 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-19 12:28:22,481	WARNING worker.py:1047 -- WARNING: 31 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-19 12:28:22,482	WARNING worker.py:1047 -- WARNING: 32 PYTHON workers have been started. This could be a 

(pid=35698) 2020-07-19 12:28:22,307	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/TrainMNIST/TrainMNIST_5_2020-07-19_12-27-44ljj1h24e/tmpruyf3jhgrestore_from_object/checkpoint
(pid=35698) 2020-07-19 12:28:22,308	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 60, '_timesteps_total': None, '_time_total': 17.396848917007446, '_episodes_total': None}


2020-07-19 12:28:23,195	WARNING worker.py:1047 -- WARNING: 33 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-19 12:28:23,229	WARNING worker.py:1047 -- WARNING: 34 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-19 12:28:23,230	WARNING worker.py:1047 -- WARNING: 35 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-19 12:28:23,232	WARNING worker.py:1047 -- WARNING: 36 PYTHON workers have been started. This could be a 

(pid=35698) 2020-07-19 12:28:23,520	INFO (unknown file):0 -- gc.collect() freed 24 refs in 0.185089112 seconds
(pid=35700) 2020-07-19 12:28:23,562	INFO (unknown file):0 -- gc.collect() freed 24 refs in 0.22155145800000042 seconds


...

(pid=36606) 2020-07-19 12:54:07,050	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/TrainMNIST/TrainMNIST_5_2020-07-19_12-52-04_4x4l6bo/tmp6227fo8krestore_from_object/checkpoint
(pid=36606) 2020-07-19 12:54:07,051	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 260, '_timesteps_total': None, '_time_total': 86.63231945037842, '_episodes_total': None}


2020-07-19 12:54:12,364	WARNING worker.py:1047 -- WARNING: 26 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=36612) 2020-07-19 12:54:14,122	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/TrainMNIST/TrainMNIST_6_2020-07-19_12-52-046qdb9hj3/tmpm4lpraoqrestore_from_object/checkpoint
(pid=36612) 2020-07-19 12:54:14,122	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 280, '_timesteps_total': None, '_time_total': 91.36839985847473, '_episodes_total': None}


2020-07-19 12:54:20,257	WARNING worker.py:1047 -- WARNING: 26 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=36616) 2020-07-19 12:54:21,956	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/TrainMNIST/TrainMNIST_6_2020-07-19_12-52-046qdb9hj3/tmpaytf7baqrestore_from_object/checkpoint
(pid=36616) 2020-07-19 12:54:21,956	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 320, '_timesteps_total': None, '_time_total': 99.53643941879272, '_episodes_total': None}


### Replacing an Existing Endpoint or Backend

## Serve is a Singleton in the Ray Cluster

You may have noticed that when defining endpoints and backends, we called Serve API methods, not methods on a Serve _class instance_. Serve is actually a [singleton](https://en.wikipedia.org/wiki/Singleton_pattern) in the whole Ray cluster, not just the driver program. 

This means that even when you terminate this notebook, our definitions above will persist! Hence, you need to clean up any endpoints and backends that are no longer needed, as follows:

In [ ]:
eps = serve.list_endpoints()
for name in eps.keys():
    serve.delete_endpoint(name)

In [ ]:
bes = serve.list_backends()
for name in bes.keys():
    serve.delete_backend(name)

In [1]:
eps = serve.list_endpoints()
bes = serve.list_backends()
print(f'endpoints: {eps}, backends {bes}')

NameError: name 'serve' is not defined

In [ ]:
serve.shutdown(name='serve-example-1') 